In [1]:
!pip install pandas

/bin/bash: pip: command not found


In [2]:
import os
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, f1_score
from torch.nn.functional import softmax


ModuleNotFoundError: No module named 'pandas'

In [ ]:
DATA_PATH = "bias_with_discourse_context.csv"
df = pd.read_csv(DATA_PATH)

# Drop missing or empty sentences
df = df.dropna(subset=["text"])
df["label"] = df["has_bias"].astype(int)


In [ ]:
df["input_text"] = (
    "Discourse: " + df["discourse_role"].fillna("NONE") +
    " | Prev: " + df["prev_discourse_role"].fillna("NONE") +
    " | Next: " + df["next_discourse_role"].fillna("NONE") +
    " | Article stance: " + df["article_bias"].fillna("UNKNOWN") +
    " | Sentence: " + df["text"].fillna("")
)

print("📊 Dataset preview:")
print(df[["article_id", "article_bias", "discourse_role", "label"]].head())


In [ ]:
dataset = Dataset.from_pandas(df[["article_id", "input_text", "label"]])

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=128)

dataset = dataset.map(tokenize, batched=True)

# Split 80/10/10
train_testvalid = dataset.train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5, seed=42)

train_ds = train_testvalid["train"]
val_ds = test_valid["train"]
test_ds = test_valid["test"]


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="./checkpoints/sentence_bias",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()


In [ ]:
metrics = trainer.evaluate(test_ds)
print("📈 Test metrics:", metrics)


In [ ]:
model.eval()
bias_probs = []

for row in df.itertuples():
    tokens = tokenizer(
        row.input_text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )
    with torch.no_grad():
        logits = model(**tokens).logits
        prob = softmax(logits, dim=-1)[0, 1].item()
    bias_probs.append(prob)

df["bias_prob"] = bias_probs


In [ ]:
article_scores = df.groupby("article_id")["bias_prob"].mean().reset_index()
article_scores["predicted_article_bias"] = article_scores["bias_prob"].apply(
    lambda x: "biased" if x > 0.55 else "neutral"
)

print("\n🧾 Article-level results:")
print(article_scores.head())

# Save outputs
df.to_csv("./sentence_bias_predictions.csv", index=False)
article_scores.to_csv("./article_bias_predictions.csv", index=False)

print("\n✅ Sentence-level and article-level predictions saved.")